# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [63]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [64]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kalemie,-5.9475,29.1947,20.73,88,100,1.24,CD,1738278155
1,1,breyten,-26.3018,29.9870,16.74,83,72,1.26,ZA,1738278133
2,2,utrik,11.2278,169.8474,26.20,71,76,5.81,MH,1738278158
3,3,el bolson,-41.9667,-71.5167,24.68,27,0,3.98,AR,1738278030
4,4,college,64.8569,-147.8028,-25.44,71,100,0.68,US,1738278160


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [65]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
city_data_df.hvplot.points(x='Lng',y='Lat',s='Humidity',geo=True,tiles=True,color="City")
# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [66]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df=city_data_df[(city_data_df['Max Temp']>21)&(city_data_df['Max Temp']<27)]
city_data_df=city_data_df[city_data_df['Cloudiness']==0]
city_data_df=city_data_df[city_data_df['Wind Speed']<4.5]
# Drop any rows with null values
# YOUR CODE HERE
city_data_df=city_data_df.dropna()
# Display sample data
# YOUR CODE HERE

### Step 3: Create a new DataFrame called `hotel_df`.

In [67]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df=city_data_df.copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name']=''

# Display sample data
# YOUR CODE HERE

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [68]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'apiKey':geoapify_key,
    'categories':'accommodation.hotel'
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = 'circle:'+str(row['Lng'])+","+str(row['Lat'])+","+str(10000)
    params["bias"] = 'proximity:'+str(row["Lng"])+','+str(row['Lat'])
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
el bolson - nearest hotel: Cabañas Willy
citrus springs - nearest hotel: Comfort Suites
saint croix - nearest hotel: The Buccaneer
santa cruz cabralia - nearest hotel: Campo Bahia Resort
cabo san lucas - nearest hotel: Comfort Rooms
inhambane - nearest hotel: Hotel de Inhambane


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
3,3,el bolson,-41.9667,-71.5167,24.68,27,0,3.98,AR,1738278030,Cabañas Willy
49,49,citrus springs,28.9975,-82.4707,23.83,50,0,1.54,US,1738277944,Comfort Suites
109,109,saint croix,17.7397,-64.7388,25.25,87,0,2.57,VI,1738278304,The Buccaneer
114,114,santa cruz cabralia,-16.2781,-39.0247,26.13,89,0,4.12,BR,1738278310,Campo Bahia Resort
142,142,cabo san lucas,22.8909,-109.9124,22.09,48,0,3.60,MX,1738278233,Comfort Rooms
155,155,inhambane,-23.8650,35.3833,24.08,83,0,2.03,MZ,1738278215,Hotel de Inhambane


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [71]:
%%capture --no-display

# Configure the map plot
hotel_df.hvplot.points(x='Lng',y='Lat',size=35,geo=True,tiles=True,color="City",hover_cols=['Country','Humidity','Hotel Name'])
# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Humidity,Hotel Name)